# #OpenIE with BERT-base

# Install && Import Packages

In [0]:
#google drive, spread sheet
!pip install --upgrade -q gspread PyDrive

    100% |████████████████████████████████| 993kB 20.1MB/s 


In [0]:
import os
import string
import re
import json
import csv
import pickle
import requests
import pandas as pd
import numpy as np
import collections
from pprint import pformat
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.preprocessing.label import LabelEncoder
import tensorflow as tf

In [0]:
tf.__version__

'1.13.0-rc1'

In [0]:
#Goole Drive libs configuration
from oauth2client.client import GoogleCredentials
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
#@title Authenticate Google Drive client && create local directory
auth.authenticate_user()
credential = GoogleCredentials.get_application_default()
#iniate Google Spread Sheet
gc = gspread.authorize(credential)
#iniate Google Drive
gauth = GoogleAuth()
gauth.credentials = credential
drive = GoogleDrive(gauth)


#Import file from Google Drive
def importfile(file_id='0Bxfl1AFkt6bcbFZXU1phSEJnVEE'):
    f = drive.CreateFile({'id': file_id})
    print('Drive file (title: %s, id: %s)' % (f['title'], f['id']))
    fname = os.path.join(os.getcwd(), f['title'])
    print('file path: {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    return fname

def exportfile(file_name="sample.txt"):
    uploaded = drive.CreateFile({'title':file_name})
    uploaded.Upload()
    print('file {} uploaded to Google Drive'.format(file_name))
    
def create_dir(dir_name="Sample"):
  dir_path=os.path.join(os.getcwd(),dir_name)
  if not os.path.exists(dir_path):
    os.makedirs(dir_name)
  return dir_path


CWD=os.getcwd()
print("Current root dir: ",CWD)
!ls

Current root dir:  /content
adc.json  bert_repo  sample_data


#TPU listing& BERT Loading

In [0]:
import pprint
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.5.59.98:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 55446565952844180),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14733192263986654729),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8161549324322787106),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10538380301613804568),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2322482719535278719),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 6180007361197668941),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 16424764579719041854),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 10024316541192518424),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1726573615257796

In [0]:
import sys
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

Cloning into 'bert_repo'...
remote: Enumerating objects: 317, done.
remote: Total 317 (delta 0), reused 0 (delta 0), pack-reused 317
Receiving objects: 100% (317/317), 253.77 KiB | 3.38 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR
BUCKET = 'dan-classification' #@param {type:"string"}
TASK = 'equip-hsc-9-classify' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/ml/bert/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [0]:
!ls bert_repo

CONTRIBUTING.md		    predicting_movie_reviews_with_bert_on_tf_hub.ipynb
create_pretraining_data.py  README.md
extract_features.py	    requirements.txt
__init__.py		    run_classifier.py
LICENSE			    run_classifier_with_tfhub.py
modeling.py		    run_pretraining.py
modeling_test.py	    run_squad.py
multilingual.md		    sample_text.txt
optimization.py		    tokenization.py
optimization_test.py	    tokenization_test.py


In [0]:
#@Import BERT moduels
import modeling
import optimization
import tokenization

# Input

1.   Example, Featurees
2.   Data Processer






In [0]:
class InputExample(object):
    def __init__(self, guid, text, label):
        self.guid=guid
        self.text=text
        self.label = label
    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = ""
        s += "guid: %s" % (self.guid)
        s += "text: %s" % (self.text)
        return s
      
class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """
  def __str__(self):
    return self.__repr__()
  def __repr__(self):
    return "PaddingInputExample"

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id):
    #required fields
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    
  def __str__(self):
    return self.__repr__()

  def __repr__(self):
    s = ""
    s += ", input_ids: %s" % (self.input_ids)
    s += ", input_mask: %s" % (self.input_mask)
    s += ", segment_ids: %s" % (self.segment_ids)
    s += ", label_id: %s" % (self.label_id)

    return s

class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_data(cls, input_file, range_start, range_end):
        """Read labels and sentences from Google Sheet."""
        worksheet = gc.open(input_file).sheet1
        labels = [x.value.strip() for x in worksheet.range("B" + range_start + ":B" + range_end) if x.value != ""]
        text = [x.value.strip() for x in worksheet.range("A" + range_start + ":A" + range_end) if x.value != ""]

        return (labels, text)
   
      
class DescClassProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        return self._create_example(
            self._read_data("equipment-hsc", "1", "300000"), "train"
        )

    def get_dev_examples(self, data_dir):
        return ""

    def get_test_examples(self,data_dir):
        return ""

    def get_labels(self,data_dir):
      labels={"label_list": None, "label_map": None}

      if os.path.exists(os.path.join(data_dir,'labels.pkl')):
        with open(os.path.join(data_dir,'labels.pkl'),'rb') as rf:
          labels = pickle.load(rf)
      else:  
        encoder = LabelEncoder()
        worksheet = gc.open("equipment-hsc").sheet1
        train_labels = [x.value.strip() for x in worksheet.range("B1:B300000") if x.value != ""]
      
        encoder.fit(train_labels)
        classes=encoder.classes_
        labels["label_list"] = classes
      
        label_map = {}
        for (i, label) in enumerate(classes):
          label_map[label] = i
        
        labels["label_map"] = label_map
      
        with open(os.path.join(data_dir,'labels.pkl'),'wb') as w:
          pickle.dump(labels,w)
      
      return labels["label_list"], labels["label_map"]
    
    def _create_example(self, data, set_type):
        labels, sentences = data
        examples = []
        for (i, sent) in enumerate(sentences):
            guid = "%s-%s" % (set_type, i)
            text = tokenization.convert_to_unicode(sent)
            label = tokenization.convert_to_unicode(labels[i])
            
            examples.append(InputExample(guid=guid, text=text, label=label))
        return examples


class FeatureWriter(object):
  """Writes InputFeature to TF example file."""

  def __init__(self, filename, is_training):
    self.filename = filename
    self.is_training = is_training
    self.num_features = 0
    self._writer = tf.python_io.TFRecordWriter(filename)

  def process_feature(self, feature):
    """Write a InputFeature to the TFRecordWriter as a tf.train.Example."""
    self.num_features += 1

    def create_int_feature(values):
      feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return feature

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_int_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    features["label_ids"] = create_int_feature([feature.label_id])

    # if self.is_training:

    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    self._writer.write(tf_example.SerializeToString())

  def close(self):
    self._writer.close()


In [0]:
def convert_single_example_to_feature(example_index, example, label_list, tokenizer, max_seq_length, is_training, label_map=None):
  
  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0)
  
  
  if label_map == None:
    label_map={}
    for (i, label) in enumerate(label_list):
      label_map[label] = i

  tokens = tokenizer.tokenize(example.text)
  # Account for [CLS] and [SEP] with "- 2"
  if len(tokens) > max_seq_length - 2:
    tokens = tokens[0:(max_seq_length - 2)]

  bert_tokens = []
  segment_ids = []
  bert_tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens:
    bert_tokens.append(token)
    segment_ids.append(0)
  bert_tokens.append("[SEP]")
  segment_ids.append(0)
  
  input_ids = tokenizer.convert_tokens_to_ids(bert_tokens)
  
  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)
  
  # Zero-pad up to the sequence length
  while len(input_ids) < max_seq_length:
    bert_tokens.append("[PAD]")
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)
    
 
  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if example_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in bert_tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))
  elif example_index % 10000 == 0:
    tf.logging.info(example_index)

  feature = InputFeatures(
    input_ids=input_ids,
    input_mask=input_mask,
    segment_ids=segment_ids,
    label_id=label_id
  )
  
  return feature
  
def convert_examples_to_features(examples, label_list, tokenizer, max_seq_length, is_training, output_fn, label_map=None):
  """Loads a data file into a list of `InputBatch`s."""
  features=[]
  for (example_index, example) in enumerate(examples):
    feature=convert_single_example_to_feature(example_index, example,label_list, tokenizer, max_seq_length, is_training, label_map)
    features.append(feature)
    # Run callback
    output_fn(feature)
  
  return features
  

#Flags& Parameters

In [0]:
class flags(object):
  pass

FLAGS = flags()

""" Required parameters"""
# "The config json file corresponding to the pre-trained BERT model. 
FLAGS.bert_config_file= os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
# flags.DEFINE_string("vocab_file", None,"The vocabulary file that the BERT model was trained on.")
FLAGS.vocab_file=os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
FLAGS.data_dir = CWD
#     "The output directory where the model checkpoints will be written.")
FLAGS.output_dir=OUTPUT_DIR#create_dir("oie_out")

#"Initial checkpoint (usually from a pre-trained BERT model"
FLAGS.init_checkpoint=os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
#     "Whether to lower case the input text. Should be True for uncased "
FLAGS.do_lower_case=True
#     "The maximum total input sequence length after WordPiece tokenization. "
#     "Sequences longer than this will be truncated, and sequences shorter "
#     "than this will be padded.")
FLAGS.max_seq_length=64

"""Training&Evalutaion&Prediction Configuration"""
FLAGS.do_train=True
FLAGS.do_eval=False
FLAGS.do_predict=True
# flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")
FLAGS.train_batch_size=64
FLAGS.eval_batch_size=64
FLAGS.predict_batch_size=64
# flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")
FLAGS.learning_rate=5e-5
# flags.DEFINE_float("num_train_epochs", 3.0,"Total number of training epochs to perform.")
FLAGS.num_train_epochs=40.0
# flags.DEFINE_float("warmup_proportion", 0.1,"Proportion of training to perform linear learning rate warmup for. "
#     "E.g., 0.1 = 10% of training.")
FLAGS.warmup_proportion=0.1
# flags.DEFINE_integer("save_checkpoints_steps", 100,"How often to save the model checkpoint.")
FLAGS.save_checkpoints_steps=1000
# flags.DEFINE_integer("iterations_per_loop", 100,"How many steps to make in each estimator call.")
FLAGS.iterations_per_loop=1000
# flags.DEFINE_integer("n_best_size", 20,"The total number of n-best predictions to generate in the nbest_predictions.json output file.")
FLAGS.n_best_size=20


"""TPU Usage Configuration"""
# flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
FLAGS.use_tpu=True
FLAGS.tpu_name=TPU_ADDRESS
#     "[Optional] GCE zone where the Cloud TPU is located in. If not "
FLAGS.tpu_zone=None
#[Optional] Project name for the Cloud TPU-enabled project. If not "
FLAGS.gcp_project=None
#[Optional] TensorFlow master URL.")
FLAGS.master=None
# "num_tpu_cores", 8,"Only used if `use_tpu` is True. Total number of TPU cores to use.")
FLAGS.num_tpu_cores=8
#     "If true, all of the warnings related to data processing will be printed. "
FLAGS.verbose_logging=False

FLAGS.task_name="equip-hsc-9-classify"

#softmax_cross_entropy_with_logits_v2

In [0]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 use_one_hot_embeddings, labels, num_labels):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  output_layer = model.get_pooled_output()

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())
  
  with tf.variable_scope("loss"):
    if is_training:
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        
    #output_layer = tf.reshape(output_layer, [-1, hidden_size])
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    
    
    ###########Using softmax_cross_entropy_with_logits_v2#################
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
    
    tf.logging.info("one_hot_labels: %s" % one_hot_labels.shape)
    tf.logging.info("logits: %s" % logits.shape)
    
    per_example_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_labels,logits = logits)
    loss = tf.reduce_mean(per_example_loss)
    
    return (loss, per_example_loss, logits, probabilities)

def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings,num_labels):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]
    

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        bert_config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings,
        labels=label_ids,
        num_labels=num_labels)

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions={"probabilities": probabilities}, scaffold_fn=scaffold_fn)
    
    elif mode == tf.estimator.ModeKeys.EVAL:
      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        acc = tf.metrics.accuracy(labels=label_ids, predictions=predictions)
        loss = tf.metrics.mean(values=per_example_loss)
        
        return {
            "eval_accuracy": acc,
            "eval_loss": loss
        }
      
      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics,
          scaffold_fn=scaffold_fn)
    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

def input_fn_builder(input_file, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""
  
  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn


def validate_flags_or_throw(bert_config):
  """Validate the input FLAGS or throw an exception."""
  if not FLAGS.do_train and not FLAGS.do_eval:
    raise ValueError("At least one of `do_train` or `do_eval` must be True.")

  if FLAGS.max_seq_length > bert_config.max_position_embeddings:
    raise ValueError(
        "Cannot use sequence length %d because the BERT model "
        "was only trained up to sequence length %d" %
        (FLAGS.max_seq_length, bert_config.max_position_embeddings))


# Build the Model

In [0]:
!rm labels.pkl

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
  
#import BERT Model config
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)
validate_flags_or_throw(bert_config)

#define dataset processors
processors = {
  "equip-hsc-10-classify": DescClassProcessor
}
#taks_name: oie: OpenIE
task_name = FLAGS.task_name.lower()
if task_name not in processors:
    raise ValueError("Task not found: %s" % (task_name))
#init task speficied dataset processor
processor = processors[task_name]()

#get the label_list from dataset
label_list, label_map = processor.get_labels(data_dir=FLAGS.data_dir)
print("positive labels: ",label_list)


#init BERT tokenizer
tokenizer = tokenization.FullTokenizer(
  vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

#config TPU
tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(FLAGS.tpu_name)
      #, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
#     master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

positive labels:  ['-31431293' '-32411AF1' '-32411AF2' ... 'agt2008' 'agt521113' 'runhourlo']


In [0]:
train_examples = None
num_train_steps = None
num_warmup_steps = None

import random
if FLAGS.do_train:
    train_examples = processor.get_train_examples(data_dir=FLAGS.data_dir)
    num_train_steps = int(
        len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
    num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

    rng = random.Random(12345)
    rng.shuffle(train_examples)
    
model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu,
    num_labels=len(label_list))#0 means: un-predicated labels

# If TPU is not available, this will fall back to normal Estimator on CPU or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': 'gs://dan-classification/ml/bert/models/equip-hsc-10-classify', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.5.59.98:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd5110c9b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.5.59.98:8470', '_evaluation_master': 'grpc://10.5.59.98:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_c

# Train estimator

In [0]:
if FLAGS.do_train:
  # We write to a temporary file to avoid storing very large constant tensors
  # in memory.
  train_writer = FeatureWriter(filename=os.path.join(FLAGS.output_dir, "train.tf_record"),is_training=True)

  convert_examples_to_features(
      examples=train_examples,
      label_list=label_list,
      label_map=label_map,
      tokenizer=tokenizer,
      max_seq_length=FLAGS.max_seq_length,
      is_training=True,
      output_fn=train_writer.process_feature)
  train_writer.close()

  tf.logging.info("***** Running training *****")
  tf.logging.info("  Num orig examples = %d", len(train_examples))
  tf.logging.info("  Num split examples = %d", train_writer.num_features)
  tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
  tf.logging.info("  Num steps = %d", num_train_steps)

  del train_examples

  train_input_fn = input_fn_builder(
      input_file=train_writer.filename,
      seq_length=FLAGS.max_seq_length,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-84894
INFO:tensorflow:tokens: [CLS] 58 fire damp ##er 2 - 15 - 1 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:tensorflow:input_ids: 101 5388 2543 10620 2121 1016 1011 2321 1011 1015 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 5131131H1 (id = 14197)
INFO:

In [0]:
FLAGS.max_seq_length

128

# Exporting

In [0]:
def serving_input_fn(batch_size=8):
  input_ids = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'input_ids')
  label_ids = tf.placeholder(tf.int64, (batch_size,), 'label_ids')
  input_mask = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'input_mask')
  segment_ids = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'segment_ids')
  input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
      'label_ids': label_ids,
      'input_ids': input_ids,
      'input_mask': input_mask,
      'segment_ids': segment_ids,
  },batch_size)()
  return input_fn

In [0]:
#@title Exporting to SavedModel for Tensorflow Serving

def build_serving_input_receiver_fn(seq_length=FLAGS.max_seq_length, batch_size=128):
  """
  This is used to define inputs to serve the model.
  :return: ServingInputReciever
  """
  feature_spec = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64)
  }
  serialized_tf_example = tf.placeholder(shape=[batch_size], dtype=tf.string)
  receiver_tensors = {'examples': serialized_tf_example}
  features = tf.parse_example(serialized_tf_example, feature_spec)
  return tf.estimator.export.ServingInputReceiver(features,receiver_tensors)



def serving_input_fn(batch_size=8):
  input_ids = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'input_ids')
  label_ids = tf.placeholder(tf.int64, (batch_size,), 'label_ids')
  input_mask = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'input_mask')
  segment_ids = tf.placeholder(tf.int64, (batch_size, FLAGS.max_seq_length), 'segment_ids')
  input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn({
      'label_ids': label_ids,
      'input_ids': input_ids,
      'input_mask': input_mask,
      'segment_ids': segment_ids,
  },batch_size)()
  return input_fn

serving_input_receiver_fn=build_serving_input_receiver_fn(FLAGS.max_seq_length,128)
estimator._export_to_tpu = False  # this is important
estimator.export_savedmodel(
    export_dir_base=os.path.join(FLAGS.output_dir, "pb"),
    serving_input_receiver_fn=build_serving_input_receiver_fn
    )

TypeError: ignored

In [0]:
!saved_model_cli show --dir 'gs://dan-classification/ml/bert/models/description-class-s-classify/pb/1548449937' --all

#!saved_model_cli show --dir 'gs://chirrpailab/ml/bert/models/oie/pb/1547140887' --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['examples'] tensor_info:
        dtype: DT_STRING
        shape: (128)
        name: Placeholder:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (128, 23)
        name: loss/Softmax:0
  Method name is: tensorflow/serving/predict


# Evaluation estimator

In [0]:
eval_examples = processor.get_train_examples(FLAGS.data_dir)
eval_writer = FeatureWriter(filename=os.path.join(FLAGS.output_dir, "eval.tf_record"),is_training=False)
convert_examples_to_features(
  examples=eval_examples,
  label_list=label_list,
  label_map=label_map,
  tokenizer=tokenizer,
  max_seq_length=FLAGS.max_seq_length,
  is_training=False,
  output_fn=eval_writer.process_feature)
eval_writer.close()

# If running eval on the TPU, you will need to specify the number of steps.
eval_steps = None
if FLAGS.use_tpu:
  eval_steps = int(len(eval_examples) / FLAGS.eval_batch_size)
eval_drop_remainder = True if FLAGS.use_tpu else False

tf.logging.info("***** Running Evaluation *****")
tf.logging.info("  Num evaluation examples = %d", len(eval_examples))
#tf.logging.info("  Num split examples = %d", len(eval_features))
tf.logging.info("  Evaluation batch size = %d", FLAGS.eval_batch_size)
tf.logging.info(" Evaluation steps = %d", eval_steps)

eval_input_fn = input_fn_builder(
    input_file=eval_writer.filename,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=eval_drop_remainder)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

output_eval_file = os.path.join(FLAGS.output_dir, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
  tf.logging.info("***** Eval results *****")
  for key in sorted(result.keys()):
    tf.logging.info("  %s = %s", key, str(result[key]))
    writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-0
INFO:tensorflow:tokens: [CLS] 72 no . 3 main ds ##l eng no . 12 cylinder [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:tensorflow:input_ids: 101 5824 2053 1012 1017 2364 16233 2140 25540 2053 1012 2260 7956 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 2331311C (id = 2793)
INFO:

# Prediction estimator

In [0]:
import csv

!rm failed.csv
# Add headers to csv
with open('failed.csv', 'w', newline='') as csvfile:
    fail_writer = csv.writer(csvfile)
    fail_writer.writerow(['Prediction', 'Actual', 'Sentence'])

def append_to_fail_csv(prediction, actual):
  with open('failed.csv', 'a', newline='') as csvfile:
    fail_writer = csv.writer(csvfile)
    fail_writer.writerow([pred, actual.label, actual.text])
  

In [0]:
if FLAGS.do_predict:
  predict_examples = processor.get_test_examples(data_dir=FLAGS.data_dir)
  predict_writer = FeatureWriter(filename=os.path.join(FLAGS.output_dir, "predict.tf_record"),is_training=False)

  convert_examples_to_features(
      examples=predict_examples,
      label_list=label_list,
      label_map=label_map
      tokenizer=tokenizer,
      max_seq_length=FLAGS.max_seq_length,
      is_training=False,
      output_fn=predict_writer.process_feature)
  predict_writer.close()

  tf.logging.info("***** Running training *****")
  tf.logging.info("  Num examples = %d", len(predict_examples))
  tf.logging.info("  Batch size = %d", FLAGS.predict_batch_size)

  predict_input_fn = input_fn_builder(
      input_file=predict_writer.filename,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=True)
  result = estimator.predict(input_fn=predict_input_fn)
  
  i = 0
  correct = 0
  for prediction in result:
    # Account for the empty label we added to the total number during training
    pred = label_list[np.argmax((prediction['probabilities']))]
    actual = predict_examples[i]
    print('#{}'.format(i), '|', pred, '|', actual.label, '|', (pred == actual.label), '|', actual.text)
    i += 1
    correct += int(pred == actual.label)
    if (int(pred == actual.label) == 0):
      append_to_fail_csv(pred, actual)
    
  print(correct, '/', i)
  print(correct / i)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-0
INFO:tensorflow:tokens: [CLS] 54 diesel doctor test engine no . 1 cylinder [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:tensorflow:input_ids: 101 5139 7937 3460 3231 3194 2053 1012 1015 7956 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 14 (id = 5)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] 72 battery preservation refrigerator [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:tensorflow:input_ids: 101 5824 6046 8347 18097 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
uploaded = drive.CreateFile({'title': 'failed.csv'})
uploaded.SetContentFile('failed.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1hTTgQsLp86QECPmdlU9JC3jRG95A_g_Q


# Prediction DataProcessor

In [0]:
import re


class DocProcessor():
  @classmethod
  def batch_examples(self,examples, batch_size): 
    # looping till length l 
    for i in range(0, len(examples), batch_size):
      batch = examples[i:i + batch_size]
      paddding_size=batch_size-len(batch)
      if(paddding_size)>0:
        batch.extend([PaddingInputExample()]*paddding_size)
      yield batch
    
  def _create_example(self, sentences, set_type="prediction"):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, sent) in enumerate(sentences):
      guid = "%s-%s" % (set_type, i)
      sent_tokens = sent["sent_tokens"]
      sent_str = tokenization.convert_to_unicode(sent["sent_str"])
      labels=[]
      if(set_type=="prediction"):
        labels=["X"]*len(sent["sent_tokens"])
       

      examples.append(InputExample(guid=guid, run_id=guid, sent_tokens=sent_tokens, sent_str=sent_str, labels=labels))
    return examples
    
  def _get_examples_from_doc(self, doc):
    #split sentences from a document
    lines=re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',doc)
    #sentences = [for sent in sentences]
    sentences=[]
    for (i, sent) in enumerate(lines):
      sent_str = tokenization.convert_to_unicode(sent.strip())
      sent_tokens = sent_str.split()
      #piece_tokens=tokenizer.tokenize(sent_str)
      sentences.append({"sent_str":sent.strip(),"sent_tokens": sent_tokens}) 
    return self._create_example(sentences)
  
  def get_labels(self,data_dir="/content/"):
      labels={"label_list":None,"label_map":None}
      if os.path.exists(os.path.join(data_dir,'labels.pkl')):
        with open(os.path.join(data_dir,'labels.pkl'),'rb') as rf:
          labels = pickle.load(rf)
      else:
        encoder = LabelEncoder()
        df = pd.read_csv(os.path.join(data_dir,"hsc_ner.train.csv"),sep = ",", header = 0, keep_default_na = False)
        df = df.drop(df[df.ENT_TYPE == ""].index)
        df['label'] = df[['ENT_TYPE', 'ENT_BIO']].apply(lambda x: 
                                                '{}-{}'.format(x[0],x[1]) 
                                                if x[1]!="O" and x[0] !=""
                                                else x[1], axis=1)
        encoder.fit(df["label"].values)
        classes=list(np.insert(encoder.classes_,0,["X","[CLS]","[SEP]"]))
        labels["label_list"]=classes

        label_map = {}
        for (i, label) in enumerate(classes,1):
            label_map[label] = i

        labels["label_map"]=label_map
        
        with open(os.path.join(data_dir,'labels.pkl'),'wb') as w:
            pickle.dump(labels,w)

      return labels["label_list"]
  

  
dp = DocProcessor()  
 
doc=r"Inspection Results : The surveyor Bill Tye attended the Tow Conference and surveyed the vessel Monday 12 March 2007.  There are several outstaying items that prevented the issuance of the trip-in-to survey approval.  There was ballast still being shifted to trim for towing, alarms had not been set according to Jim Ruth's specifications, condition Zebra had not been set for towing; subsequently Bill Tye was not able to issue an approved survey.  Monday 19 March 2007 Bill Tye will again re-survey the outstanding items and approve the tow if it is ready next Monday morning.  Only one trip by the surveyor was quoted in the cost to approve the tow.  The additional costs for the surveyor to attend this coming Monday, 19 March 2007, need to be approved and will be additional costs beyond the quote.  Please approve these additional costs so we can proceed with the tow. Recommendations :\n \n Compensate the contractor for additional cost to re-survey the vessel prior to tow."    
sent_examples =dp._get_examples_from_doc(doc)

In [0]:
batched_dataset = list(DocProcessor.batch_examples(sent_examples, 32))
batched_dataset

In [0]:
label_list= dp.get_labels("/content/")
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=True)

In [0]:
examples =[]
def process_feature(feature):
  
  def create_int_feature(values):
      feature = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return feature
  
  features = collections.OrderedDict()
  features["input_ids"] = create_int_feature(feature.input_ids)
  features["input_mask"] = create_int_feature(feature.input_mask)
  features["segment_ids"] = create_int_feature(feature.segment_ids)
  features["label_ids"] = create_int_feature(feature.label_ids)
  
  features=tf.train.Features(feature=features)
  serialized_example=tf.train.Example(features=features).SerializeToString()
  examples.append(serialized_example)

#predict_file = os.path.join(FLAGS.output_dir, "predict.tf_record")
# predict_writer = FeatureWriter(filename=predict_file,is_training=False)
input_features=convert_examples_to_features(examples=batched_dataset[0], 
    label_list=label_list,
    tokenizer=tokenizer,
    max_seq_length=128,
    is_training=False,
    output_fn=process_feature)

INFO:tensorflow:*** Example ***
INFO:tensorflow:run_id: prediction-0
INFO:tensorflow:tokens: [CLS] inspection results : the surveyor bill ty ##e attended the tow conference and surveyed the vessel monday 12 march 2007 . [SEP] **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** **NULL** 

[b'\n\xf5\x04\n\x95\x01\n\ninput_mask\x12\x86\x01\x1a\x83\x01\n\x80\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\n\xaa\x01\n\tinput_ids\x12\x9c\x01\x1a\x99\x01\n\x96\x01e\xc9R\x87\x1b\x80\x08\xcc\x0f\x91\x86\x01\xcd\x17\xb3.\x8f\x10\x9e\x19\xcc\x0f\xbd{\xda\x17\xce\x0f\xccd\xcc\x0f\xf20\x906\xd4\x11\xb9\x11\xf1\x11\xf4\x07f\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [0]:
list_orig_to_tok_index= [input_ft.orig_to_tok_index for input_ft in input_features]

result_list = list(zip(sent_examples, list_orig_to_tok_index))#list(zip(sent_examples, list_orig_to_tok_index, predictions))

In [0]:
for (result_index, result) in enumerate(result_list):
  print(result_index)

padding the examples into bach_size